# Webhook Example

To work with webhooks we need two things:
1. redirection of a public URI used for a webhook to the local machine. This can be achieved by using `ngrok` 
2. a local webserver to serve the POSTs to the webhook URL

## Automating the use of ngrok to tunnel webhooks to local machine


Starting `ngrok` using `ngrok http 80 -log=stdout -log-format=json -log-level=info` creates the following JSON output on stdout:

```
{"lvl":"info","msg":"no configuration paths supplied","t":"2017-06-.."}
{"err":"stat /Users/jkrohn/.ngrok2/ngrok.yml: no such file or directory","lvl":"info","msg":"ignoring default config path, could not stat it","path":"/Users/jkrohn/.ngrok2/ngrok.yml","t":"2017-06-.."}
{"comp":"memory storage","lvl":"info","msg":"run component","obj":"controller","t":"2017-06-.."}
{"comp":"signal handler","lvl":"info","msg":"run component","obj":"controller","t":"2017-06-.."}
{"comp":"Tunnel session","lvl":"info","msg":"run component","obj":"controller","t":"2017-06-.."}
{"comp":"updater","lvl":"info","msg":"run component","obj":"controller","t":"2017-06-.."}
{"comp":"web","lvl":"info","msg":"run component","obj":"controller","t":"2017-06-.."}
{"addr":"127.0.0.1:4040","lvl":"warn","msg":"can't bind default web address, trying alternatives","obj":"web","t":"2017-06-.."}
{"addr":"127.0.0.1:4041","lvl":"info","msg":"starting web service","obj":"web","t":"2017-06-.."}
{"lvl":"info","msg":"tunnel session started","obj":"tunSess","t":"2017-06-.."}
{"id":"44a534c2b867","lvl":"info","msg":"client session established","obj":"csess","t":"2017-06-.."}
```

The idea to automate the use of `ngtok` for Python is to start an `ngrok` instance from Python as a subprocess and then parse the above startup messages to:
* learn the port of the `ngrok` admin interface (in case multiple `ngrok` instances are running on the same host). This port is included in the `addr` part of this message:
`{"addr":"127.0.0.1:4041","lvl":"info","msg":"starting web service","obj":"web","t":"2017-06-.."}`
* wait until the client session with the `ngrok` service has been established. This is indicated by this message:
`{"id":"44a534c2b867","lvl":"info","msg":"client session established","obj":"csess","t":"2017-06-.."}`

The output of the  `ngrok` process needs to be read continuously to avoid that the process locks up. This needs to be  done in a separate thread.

In [1]:
import threading
import shutil
import subprocess
import requests
import time

class Ngrok(threading.Thread):
    ''' Ngrok: class to automate starting a local ngrok instance as a subprocess.
    '''
    
    def __init__(self, port=None):
        '''Initalize Ngrok tunnel.

        :param port: int, localhost port forwarded through tunnel

        '''
        assert shutil.which("ngrok"), "ngrok command must be installed, see https://ngrok.com/"
        threading.Thread.__init__(self)
        
        self.port = port
        return
    
    def read_json_from_ngrok(self):
        ''' read stdout of ngrok process and try to parse as JSON
        
            returns: decoded JSON of single ngrok output line
        '''
        while True:
            line = self.ngrok.stdout.readline().decode()
            # try to parse as JSON
            try:
                line = json.loads(line)
            except json.JSONDecodeError:
                # ignore anything that isn't JSON
                continue
            break
        return line
        
    def start(self):
        ''' 
        start a local ngrok process
        start a thread in the background to read stdout of the ngrok process
        As soon as the nrgok client connection has been established determine the public URL and report that back
        '''
        
        # where is ngrok?
        ngrok = shutil.which('ngrok')
        
        # commandline to start ngrok
        cmd = '{} http {} -log=stdout -log-format=json -log-level=info'.format(ngrok, self.port)
        
        # start ngrok process
        self.ngrok = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)

        # give it some time
        time.sleep(.5)
        assert self.ngrok.poll() is None, "ngrok failed to start"
        
        ngrok_addr = None
        print('Waiting for ngrok startup...')
        while True:
            line = self.read_json_from_ngrok()
            
            # {"addr":"127.0.0.1:4041","lvl":"info","msg":"starting web service","obj":"web","t":"2017-06-.."}
            if (line.get('obj') == 'web' and 
                line.get('lvl') == 'info' and 
                line.get('msg') == 'starting web service'):
                # get ngrok admin interface address from message
                ngrok_addr = line['addr']
            # {"id":"44a534c2b867","lvl":"info","msg":"client session established","obj":"csess","t":"2017-06-.."}
            if (line.get('obj') == 'csess' and 
                line.get('lvl') == 'info'):
                # startup done: terminate loop
                break

        # now as the ngrok client is up we can try to use the ngrok client API to get the public address of the tunnel
        # might take some time for the tunnels to come up; hence we try repeatedly until the API call succeeds
        
        ''' the expected JSON result looks something like this:
            {
                "tunnels": [
                    {
                        "proto": "https",
                        "name": "command_line",
                        "config": {
                            "addr": "localhost:60176",
                            "inspect": true
                        },
                        "metrics": {
                            ...
                        },
                        "public_url": "https://47bff724.ngrok.io",
                        "uri": "/api/tunnels/command_line"
                    },
                    {
                        "proto": "http",
                        "name": "command_line (http)",
                        "config": {
                            "addr": "localhost:60176",
                            "inspect": true
                        },
                        "metrics": {
                            ...
                        },
                        "public_url": "http://47bff724.ngrok.io",
                        "uri": "/api/tunnels/command_line+%28http%29"
                    }
                ],
                "uri": "/api/tunnels"
            }
        
        '''
        while True:
            print('Trying to get tunnel information from ngrok client API')
            response = requests.get('http://{}/api/tunnels'.format(ngrok_addr), headers={'content-type':'application/json'}).json()
            if response.get('tunnels'):
                break
            time.sleep(0.5)
        
        # Default: take the 1st URL    
        url = response['tunnels'][0]['public_url']
        
        # but we prefer HTTPS if an HTTPS tunnel exists
        https_tunnel = next((t for t in response['tunnels'] if t['proto'] == 'https'), None)
        if https_tunnel is not None:
            url = https_tunnel['public_url']
            
        # Now start the actual Thread
        threading.Thread.start(self) 
        
        # return the public URL
        return url

    def stop(self):
        """Tell ngrok to tear down the tunnel.

        Stop the background tunneling process.
        """
        self.ngrok.terminate()
        return
        
    def run(self):
        # continuously read from the ngrok process output to prevent the process from blocking
        while True:
            self.read_json_from_ngrok()
        return       

## Simple HTTP server for webhook

To be able to act on messages posted to the webhook we need a simple local HTTP server. We run this HTTP server in it's own thread. For each POST received on any of the registered URLs the HTTP server just calls the registered callback.

In [2]:
import json
from http.server import HTTPServer
from http.server import BaseHTTPRequestHandler
import uuid

class WHHandler(BaseHTTPRequestHandler):

    def do_POST(self):
        path = self.path[1:]  # w/o the leading slash
        content_len = int(self.headers.get('content-length', 0))
        body = self.rfile.read(content_len).decode()
        try:
            json_data = json.loads(body)
        except json.JSONDecodeError:
            json_data = None
        # now try to call the callback for the service
        self.server.wh_server.handle_post(path, json_data)    
        self.send_response(200, "OK")
        self.end_headers()

    def log_message(self, fmt, *args):
        ''' no logging 
        '''
        pass
    
class WHServer(threading.Thread):

    def __init__(self):
        threading.Thread.__init__(self)
        self.daemon = True
        self.httpd = None
        self.services = {}
        self.running = threading.Event()
        return

    def run(self):
        # new HTTP server on next available port
        self.httpd = HTTPServer(('', 0), WHHandler)
        self.httpd.wh_server = self
        self.running.set()
        self.httpd.serve_forever()
        return

    def stop(self):
        if self.httpd is not None:
            self.httpd.shutdown()
        return
    
    def get_server_address(self):
        self.running.wait()
        return self.httpd.server_address
    
    def add_service(self):
        ''' add a new service on the WH server
        returns the path (random UUID) of the new service'''
        while True:
            # new random URL
            u = str(uuid.uuid4())
            
            # chances are VERY low that we have a collision, but ...
            if u not in self.services:
                break
        
        # add new service w/ no callbacks
        self.services[u] = {}
        return u
    
    def remove_service(self, uri):
        self.services.pop(uri, None)
        return
        
    def add_callback(self, uri, callback):
        if uri not in self.services:
            raise KeyError('uri not registered')
        self.services[uri]['callback'] = callback
        return
    
    def handle_post(self, path, data):
        if path not in self.services:
            return
        # call the registered callback
        # if no callback is registered then the the default callback is to to nothing
        self.services[path].get('callback', lambda x: None)(data)
        return

## Putting the pieces together

First a bot needs to be registered at the [Cisco Spark for Developers Portal](https://developer.ciscospark.com/). As part of the bot registration an `access token` is created. This access toeken need to be used by the Bot code for all authenticated requests to the Spark APIs. The bot `access token` has a virtually unlimited lifetime and this does not need to be refreshed

In [7]:
import queue
import ciscosparkapi
import json
import os

# insert your Bot access token here. This token is created at developer.cisco.com under 'My Apps'
BOT_ACCESS_TOKEN = os.environ.get('BOT_ACCESS_TOKEN')
if BOT_ACCESS_TOKEN is None:
    BOT_ACCESS_TOKEN = '<insert your token here>'

def handle_messages_created(api, bot, msg):
    ''' handle a webhook notification on created messages
    
    :param api: CiscoSparkApi, Spark API instance of the bot
    :param bot: ciscosparkapi.Person, person details of bot
    :param msg: dict, JSON data received via webhook
    
    Space title and message content are not sent over the webhook in the clear to not expose private data.
    
    An exampe JSON message received over the webhook when a message is created:
        {
            "name": "Firehose",
            "orgId": "Y2lzY29zcGFyazo...lMGUxMGY",
            "actorId": "Y2lzY29zcGFy...MyOTk",
            "ownedBy": "creator",
            "event": "created",
            "status": "active",
            "appId": "Y2lzY29zcGF...5MWFh",
            "data": {
                "created": "2017-06...",
                "id": "Y2lzY29...MjY5",
                "roomId": "Y2lzY29zc...NWQx",
                "personEmail": "jkrohn@cisco.com",
                "personId": "Y2lzY2...yOTk",
                "mentionedPeople": [
                    "Y2lzY29zc...mMGQ"
                ],
                "roomType": "group"
            },
            "createdBy": "Y2lzY29zcG...mMGQ",
            "id": "Y2lzY29z...jNDJm",
            "targetUrl": "https://335097e9.ngrok.io/619e38a8-9beb-4dc3-99ad-12a8d0c12f32",
            "resource": "messages",
            "created": "2017-06..."
        }
    
    The 'data' object has the detail about the created message
    '''
    
    wh_data = ciscosparkapi.Webhook(msg)
    
    # ignore messages posted by the Bot itself
    if bot.id == wh_data.data.personId:
        return
    
    # also make sure that the Bot really has been mentioned
    # if the mentionedPeople attribute doesn't exist then we will just assume that we got a message in a 1:1 space
    try:
        if bot.id not in wh_data.data.mentionedPeople:
            return
    except AttributeError:
        pass
        
    # get the space information
    space = api.rooms.get(wh_data.data.roomId)
    
    # get message details
    message = api.messages.get(wh_data.data.id)
    print(message)
    
    # get person details
    person = api.people.get(wh_data.data.personId)
    
    print('Bot saw message in space \'{}\' posted by {}: {}'.format(space.title, person.displayName, message.text))
    
    answer = 'Hi <@personId:{}|{}>, thanks for your message ({}).'.format(person.id, person.displayName, message.text)
    api.messages.create(roomId=space.id, markdown=answer)
    return

# create a primitive web server on an available port
server = WHServer()
server.start()
port = server.get_server_address()[1]
print('Web server runnnig on port {}'.format(port))

# get a public URL for that port through ngrok redirection
ngrok = Ngrok(port=port)
ngrok_base_url = ngrok.start()
print('Public base URL: {}'.format(ngrok_base_url))

# create a queue to receive data posted to the webhook
message_queue = queue.Queue()

# register a base URL on the web server
wh_uri = server.add_service()

# and register a callback which put the message in above queue
server.add_callback(wh_uri, lambda x:message_queue.put(x))

public_wh_uri = '{}/{}'.format(ngrok_base_url, wh_uri)
print('Public URI for webhook: {}'.format(public_wh_uri))

# Spark API
api = ciscosparkapi.CiscoSparkAPI(access_token=BOT_ACCESS_TOKEN)

# people details of bot (id, etc.) 
bot = api.people.me()

# delete all existing webhooks (possibly from previous start)
existing_hooks = list(api.webhooks.list())
for h in existing_hooks:
    print('Deleting existing Webhook \'{}\' pointing to {}'.format(h.name, h.targetUrl))
    api.webhooks.delete(h.id)

# create new webhook pointing to 'our' public URI (via ngrok)
wh = api.webhooks.create(name='Firehose', targetUrl=public_wh_uri, resource='all', event='all')
print('Created webhook \'{}\' pointing to {}'.format(wh.name, wh.targetUrl))

# now read from the queue
print('Waiting for messages sent to webhook')
while True:
    data = message_queue.get()
    if data['resource'] == 'messages' and data['event'] == 'created':
        handle_messages_created(api, bot, data)
    message_queue.task_done()

Web server runnnig on port 63925
Waiting for ngrok startup...
Trying to get tunnel information from ngrok client API
Trying to get tunnel information from ngrok client API
Public base URL: https://53a30619.ngrok.io
Public URI for webhook: https://53a30619.ngrok.io/746656fe-b775-42ff-87f4-b3291aba9a39
Deleting existing Webhook 'Firehose' pointing to https://762c5680.ngrok.io/6a2d8d03-9563-4157-9e6b-1f0746bcb4f6
Created webhook 'Firehose' pointing to https://53a30619.ngrok.io/746656fe-b775-42ff-87f4-b3291aba9a39
Waiting for messages sent to webhook
Message:
{
  "roomId": "Y2lzY29zcGFyazovL3VzL1JPT00vOTE3NzM0OTAtMDFiZi0xMWU4LTg1YmEtNDE3NzRmYTA5NzE5",
  "personEmail": "jkrohn@cisco.com",
  "created": "2018-01-25T11:36:44.813Z",
  "id": "Y2lzY29zcGFyazovL3VzL01FU1NBR0UvMDU0MDdiZDAtMDFjNC0xMWU4LWJmYjctNzFlZDY5NWRhMWZj",
  "personId": "Y2lzY29zcGFyazovL3VzL1BFT1BMRS8wNzhkOGVjMi05Mjg5LTQ2NTUtOWE5NC0wNDNiOWVjMTMyOTk",
  "mentionedPeople": [
    "Y2lzY29zcGFyazovL3VzL1BFT1BMRS9iMTVlZGYyMi1lYWNlL

KeyboardInterrupt: 